# Image Import/Export 





Converting images between CASA image and FITS format








\
The **exportfits** and **importfits** tasks enable conversion between CASA images and FITS data.  The **exportfits** task allows you to write your CASA image to a FITS file that other packages can read, and the **importfits** task converts existing FITS files into CASA images.  While many image analysis tasks can operate on FITS files, we recommend converting to CASA images for processing and analysis purposes.

###  



### Export CASA Image to FITS (exportfits) {#sec425}

The **exportfits** task is used to export a CASA image to FITS format. The inputs are:

```


##  exportfits :: Convert a CASA image to a FITS file
imagename           =         ''        #  Name of input CASA image
fitsimage           =         ''        #  Name of output image FITS 
                                        #   file
velocity            =      False        #  Use velocity (rather than 
                                        #   frequency) as spectral axis
optical             =      False        #  Use the optical (rather than 
                                        #   radio) velocity convention
bitpix              =        -32        #  Bits per pixel
minpix              =          0        #  Minimum pixel value (if 
                                        #   minpix > maxpix, value is 
                                        #   automatically determined)
maxpix              =         -1        #  Maximum pixel value (if 
                                        #   minpix > maxpix, value is 
                                        #   automatically determined)
overwrite           =      False        #  Overwrite pre-existing
                                        #   imagename
dropstokes          =      False        #  Drop the Stokes axis?
stokeslast          =       True        #  Put Stokes axis last in
                                        #   header?
history             =       True        #  Write history to the FITS
                                        #   image?
dropdeg             =      False        #  Drop all degenerate axes (e.g.
                                        #   Stokes and/or Frequency)?
```

<div class="alert alert-warning">
**Alert:** The spectral axis of CASA images is nearly always in frequency rather than velocity. Velocities are computed only as a secondary mapping of the spectral channels with respect to a rest frequency.  If velocity units are desired and the user sets* velocity = True*, exportfits will write the spectral axis in velocity units *instead* of in frequency units.  The **exportfits** task will not output a FITS file with multiple spectral coordinate systems. 
</div>

As a simple example of an **exportfits** command, the following will write the CASA image (\'ngc5921.clean.image\') as a FITS file (\'ngc5921.clean.fits\').  In this case, the default parameter values will be adopted, so that the resulting FITS file will have the same axis order, number of pixels, etc. as the original CASA image.

```
exportfits(imagename='ngc5921.clean.image',outfile='ngc5921.clean.fits')
```

In some cases, the user may wish to use the *dropstokes,* *stokeslast*, and/or *dropdeg* parameters in order for the FITS image to be compatible with certain external applications. The *dropdeg *parameter will remove the frequency axis if it has a length of one channel, and/or it will drop the Stokes axis if that has a length of one (i.e. only one Stokes parameter is present). This would be useful, for example, for continuum data so that other programs will interpret it as a 2-D image rather than a cube.

See **[exportfits](https://casa.nrao.edu/casadocs-devel/stable/global-task-list/task_exportfits)** in the Global Task List for examples in which these and other parameters are specified.  

 



### FITS Image Import (importfits)

The **importfits** task enables the user to import a FITS image into CASA image table format. It is not essential to generate a CASA image file if you intend to simply view the image, as the CASA **viewer** can read FITS images, however we recommend importing to CASA image format for analyzing images with CASA. The inputs for **importfits** are:

```


##  importfits :: Convert an image FITS file into a CASA image
fitsimage           =         ''        #  Name of input image FITS file
imagename           =         ''        #  Name of output CASA image
whichrep            =          0        #  If fits image has multiple
                                        #   coordinate reps, choose one.
whichhdu            =          0        #  If its file contains
                                        #    multiple images, choose one.
zeroblanks          =       True        #  Set blanked pixels to zero (not NaN)
overwrite           =      False        #  Overwrite pre-existing imagename
defaultaxes         =      False        #  Add the default 4D
                                        #   coordinate axes where they are missing
defaultaxesvalues   =         []        #  List of values to assign to
                                        #   added degenerate axes when
                                        #   defaultaxes=True (ra,dec,freq,stokes)
```

As a simple example, the following command would create a CASA image named \'ngc5921.clean.image\' from the FITS file \'ngc5921.clean.fits\':

```
importfits(fitsimage='ngc5921.clean.fits',imagename='ngc5921.clean.image')
```

See **[importfits](https://casa.nrao.edu/casadocs-devel/stable/global-task-list/task_importfits)** in the Global Task List for more complex examples. 

 



### Extracting data from an image (**imval**)

\
The **imval** task will extract the values of the data and mask from a specified region of an image and place in the task return value as a Python dictionary. The inputs are:

```


##  imval :: Get the data value(s) and/or mask value in an image.
imagename  =      ''   #  Name of the input image
region     =      ''   #  Image Region.  Use viewer
box        =      ''   #  Select one or more box regions
chans      =      ''   #  Select the channel(spectral) range
stokes     =      ''   #  Stokes params to image (I,IV,IQU,IQUV)
```

Area selection using [box](#region-selection--box-) and [region](#regions--region-) is detailed above. By default, *box=\' \'* will extract the image information at the reference pixel on the direction axes. [Plane selection](#plane-selection--chans--stokes-) is controlled by *chans* and *stokes*. By default, *chans=\' \'* and *stokes=\' \'* will extract the image information in all channels and Stokes planes.\
\
For instance,

```
xval = imval('myimage', box='144,144', stokes='I' )
```

will extract the Stokes I value or spectrum at pixel 144,144, while

```
xval = imval('myimage', box='134,134.154,154', stokes='I' )
```

will extract a 21 by 21 pixel region. Extractions are returned in NumPy arrays in the return value dictionary, plus some extra elements describing the axes and selection:

```python

CASA <2>: xval = imval('ngc5921.demo.moments.integrated')

CASA <3>: xval
  Out[3]:
{'axes': [[0, 'Right Ascension'],
          [1, 'Declination'],
          [3, 'Frequency'],
          [2, 'Stokes']],
 'blc': [128, 128, 0, 0],
 'data': array([ 0.89667124]),
 'mask': array([ True], dtype=bool),
 'trc': [128, 128, 0, 0],
 'unit': 'Jy/beam.km/s'}
```

extracts the reference pixel value in this 1-plane image. Note that the \'*data\'* and \'*mask\'* elements are NumPy arrays, not Python lists. To extract a spectrum from a cube:

```python

CASA <8>: xval = imval('ngc5921.demo.clean.image',box='125,125')

CASA <9>: xval
  Out[9]:
{'axes': [[0, 'Right Ascension'],
          [1, 'Declination'],
          [3, 'Frequency'],
          [2, 'Stokes']],
 'blc': [125, 125, 0, 0],
 'data': array([  8.45717848e-04,   1.93370355e-03,   1.53750915e-03,
         2.88399984e-03,   2.38683447e-03,   2.89159478e-04,
         3.16268904e-03,   9.93389636e-03,   1.88773088e-02,
         3.01138610e-02,   3.14478502e-02,   4.03211266e-02,
         3.82498614e-02,   3.06552909e-02,   2.80734301e-02,
         1.72479432e-02,   1.20884273e-02,   6.13593217e-03,
         9.04005766e-03,   1.71429547e-03,   5.22095338e-03,
         2.49114982e-03,   5.30831399e-04,   4.80734324e-03,
         1.19265869e-05,   1.29435991e-03,   3.75700940e-04,
         2.34788167e-03,   2.72604497e-03,   1.78467855e-03,
         9.74952069e-04,   2.24676146e-03,   1.82263291e-04,
         1.98463408e-06,   2.02975096e-03,   9.65532148e-04,
         1.68218743e-03,   2.92119570e-03,   1.29359076e-03,
        -5.11484570e-04,   1.54162932e-03,   4.68662125e-04,
        -8.50282842e-04,  -7.91683051e-05,   2.95954203e-04,
        -1.30133145e-03]),
 'mask': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,  True], dtype=bool),
 'trc': [125, 125, 0, 45],
 'unit': 'Jy/beam'}
```

To extract a region from the plane of a cube:

```python
CASA <13>: xval = imval('ngc5921.demo.clean.image',box='126,128,130,129',chans='23')

CASA <14>: xval
  Out[14]:
{'axes': [[0, 'Right Ascension'],
          [1, 'Declination'],
          [3, 'Frequency'],
          [2, 'Stokes']],
 'blc': [126, 128, 0, 23],
 'data': array([[ 0.00938627,  0.01487772],
       [ 0.00955847,  0.01688832],
       [ 0.00696965,  0.01501907],
       [ 0.00460964,  0.01220793],
       [ 0.00358087,  0.00990202]]),
 'mask': array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True]], dtype=bool),
 'trc': [130, 129, 0, 23],
 'unit': 'Jy/beam'}

CASA <15>: print xval['data'][0][1]
0.0148777160794
```

In this example, a rectangular box was extracted, and you can see the order in the array and how to address specific elements.